# Steered MD in BioSimSpace

This notebook covers the setup of steered MD simulations using BioSimSpace. Simpler simulations with a single CV and multiple CV steering protocols are both included.

### Maintainers
* Adele Hardie -- adele.hardie@ed.ac.uk (AdeleHardie)

### Prerequisites
* BioSimSpace
* AMBER or GROMACS compiled with PLUMED

### Table of Contents
1. [Set up sMD - single CV](#Set-up-sMD---single-CV)
2. [Set up sMD - multi CV](#2-Set-up-sMD---multiple-CVs)
3. [Run sMD](#3-Run-sMD)

**<span style="color:black">Jupyter Cheat Sheet</span>**
- To run the currently highlighted cell and move focus to the next cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- To run the currently highlighted cell and keep focus in the same cell, hold <kbd>&#x21E7; Ctrl</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- To get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;

### Link to documentation:
You can find the full documentation at [biosimspace.org](https://biosimspace.org).

## 1 Set up sMD - single CV

Running steered MD in BioSimSpace is very similar to regular simulations already covered. It only requires some more preparation for interfacing with PLUMED, the software that takes care of biasing the Hamiltonian.

### 1.1 Setting up the system

We start by importing the required libraries:

In [ ]:
import BioSimSpace as BSS
import os
from shutil import copyfile

Load a system with BioSimSpace. This particular system is of PTP1B with the WPD loop open (from PDB entry 2HNP) and has been minimised and equilibrated.

In [ ]:
system = BSS.IO.readMolecules(["data/system.prm7", "data/system.rst7"])

### 1.2 Creating the CV

Steered MD uses a specific CV, which in this case is RMSD of the WPD loop (residues 178-184). To calculate RMSD, we specify a reference structure first:

In [ ]:
reference = BSS.IO.readMolecules("data/reference.pdb").getMolecule(0)

In [ ]:
rmsd_indices = []
for residue in reference.getResidues():
    if 178 <= residue.index() <= 184:
        for atom in residue.getAtoms():
            if atom.element() != "Hydrogen (H, 1)":
                rmsd_indices.append(atom.index())

In [ ]:
rmsd_cv = BSS.Metadynamics.CollectiveVariable.RMSD(system, reference, rmsd_indices)

### 1.3 Setting up a steered MD protocol

To create a protocol, we need to set up the steering restraints and schedule. As shown in equation 1, steered MD is defined by the expected CV value and the force constant &kappa; at some time *t*. Generally sMD has four stages:

| Stage          | Expected end CV | Force constant  |
| -------------- | --------------- | --------------- |
| 1. start       | initial value   | none            |
| 2. apply force | initial value   | specified force |
| 3. steering    | target value    | specified force |
| 4. relaxation  | target value    | none            |

Force is usually applied over a few picoseconds, and the bulk of the simulation is used for steering, i.e. stage 3. We need to specify the end times for these stages:


In [ ]:
start = 0 * BSS.Units.Time.nanosecond
apply_force = 4 * BSS.Units.Time.picosecond
steer = 150 * BSS.Units.Time.nanosecond
relax = 152 * BSS.Units.Time.nanosecond

In [ ]:
schedule = [start, apply_force, steer, relax]

The restraints specify the expected CV value and the force constant ($\kappa(t)$ and $\vec{s}_0(t)$) at each step created above.

In [ ]:
nm = BSS.Units.Length.nanometer
restraint_1 = BSS.Metadynamics.Restraint(rmsd_cv.getInitialValue(), 0)
restraint_2 = BSS.Metadynamics.Restraint(rmsd_cv.getInitialValue(), 3500)
restraint_3 = BSS.Metadynamics.Restraint(0 * nm, 3500)
restraint_4 = BSS.Metadynamics.Restraint(0 * nm, 0)

In [ ]:
protocol = BSS.Protocol.Steering(
    rmsd_cv,
    [start, apply_force, steer, relax],
    [restraint_1, restraint_2, restraint_3, restraint_4],
    runtime=152 * BSS.Units.Time.nanosecond,
)

### 1.4 A quick look at GROMACS

We have previously created a protocol for sMD, so all that is needed is to plug it into a GROMACS process

In [ ]:
process = BSS.Process.Gromacs(system, protocol)

We can have a look at the command arguments that will be used to run this simulation:

In [ ]:
process.getArgs()

The argument `-plumed plumed.dat` tells GROMACS to use PLUMED, looking at the `plumed.dat` file for instructions. This process can be run like any other process you have seen before. All the required files have been created in the `process.workDir()` by BioSimSpace.

### 1.5 Steered MD in AMBER

Just as with GROMACS, we simply need to create a process in AMBER.

<div class="alert alert-info">
<b>Note specifying the use of <code>pmemd.cuda</code>. Otherwise, the default executable to run sMD will be <code>sander</code> and GPU optimisation will not be used.</b>
</div>

In [ ]:
process = BSS.Process.Amber(
    system, protocol, exe=f'{os.environ["AMBERHOME"]}/bin/pmemd.cuda'
)

Check the configuration of the process:

In [ ]:
process.getConfig()

The lines `plumed=1` and `plumedfile="plumed.in"` are what specify that PLUMED will be used. The process can now be started to run steered MD.

## 2 Set up sMD - multiple CVs

The above setup example uses one collective variable - the RMSD of the WPD loop. However, there may be need for more complicated steering protocols, involving multiple CVs. Below we set up an sMD protocol using the previous rmsd CV, but also adding a torsion and a distance CVs (the start and end values for these come from behaviour observed in previous MD simulations).

### 2.1 Torsion CV

We will be adding the $\chi$ 1 angle of Tyr152 to the steering protocol. Tyr152 is suggested to be part of the PTP1B allosteric network. When the WPD loop is open (blue), it exists in both "up" and "down" rotamers, but when it is closed (orange), Tyr152 exists in the "down" rotamer only.

<img src="figures/tyr152.png" width=250>

In [ ]:
torsion_indices = []
for atom in system.getMolecule(0).getResidues()[152].getAtoms():
    if atom.name() in ["N", "CA", "CB", "CG"]:
        torsion_indices.append(atom.index())

In [ ]:
torsion_cv = BSS.Metadynamics.CollectiveVariable.Torsion(torsion_indices)

### 2.2 Distance CV

Another component of the allosteric network of PTP1B is the stacking of Phe196 to Phe280. These residues are $\pi$-stacked when the WPD loop is closed (orange) and apart when it is open (blue).

<img src="figures/phe196.png" width=250>

This stacking will be expressed as the distance between the C$\gamma$ atoms the the residues.

In [ ]:
distance_indices = []
for residue in system.getMolecule(0).getResidues():
    if residue.index() == 196 or residue.index() == 280:
        for atom in residue.getAtoms():
            if atom.name() == "CG":
                distance_indices.append(atom.index())
                break

distance_cv = BSS.Metadynamics.CollectiveVariable.Distance(
    distance_indices[0], distance_indices[1]
)

### 2.3 Creating the protocol

The restraints now have to include target values for all CVs involved at each point in the schedule.

In [ ]:
nm = BSS.Units.Length.nanometer
rad = BSS.Units.Angle.radian

In [ ]:
restraints = [
    [
        BSS.Metadynamics.Restraint(rmsd_cv.getInitialValue(), 0),
        BSS.Metadynamics.Restraint(1.1 * rad, 0),
        BSS.Metadynamics.Restraint(0.56 * nm, 0),
    ],  # initial
    [
        BSS.Metadynamics.Restraint(rmsd_cv.getInitialValue(), 3500),
        BSS.Metadynamics.Restraint(1.1 * rad, 3500),
        BSS.Metadynamics.Restraint(0.56 * nm, 3500),
    ],  # apply force
    [
        BSS.Metadynamics.Restraint(0 * nm, 3500),
        BSS.Metadynamics.Restraint(1.1 * rad, 3500),
        BSS.Metadynamics.Restraint(0.4 * nm, 3500),
    ],  # steering
    [
        BSS.Metadynamics.Restraint(0 * nm, 0),
        BSS.Metadynamics.Restraint(1.1 * rad, 0),
        BSS.Metadynamics.Restraint(0.4 * nm, 0),
    ],
]  # release force

When creating the `protocol`, all CVs are passed as a list.

<div class="alert alert-info">
<b>The order of the CVs when creating the protocol needs to match the order of the restraints above.</b>
</div>

In [ ]:
protocol = BSS.Protocol.Steering(
    [rmsd_cv, torsion_cv, distance_cv],
    schedule,
    restraints,
    runtime=relax,
    report_interval=2500,
    restart_interval=2500,
)

In [ ]:
process = BSS.Process.Amber(
    system, protocol, exe=f'{os.environ["AMBERHOME"]}/bin/pmemd.cuda'
)

We can check the contents of the PLUMED file that BioSimSpace has created:

In [ ]:
for line in process.getPlumedConfig():
    print(line)

The `ARG` line specifies 3 CVs, which will all be used for steering.

## 3 Run sMD

The process can be run in the notebook:

In [ ]:
process.start()

Otherwise, there are example python scripts provided to run both the [single](scripts/sMD_simple.py) and [multi](scripts/sMD_multiCV.py) CV sMD simulations as outlined above.